In [1]:
import os
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

from time import sleep

import sistemas
import sei
import functions
import page
from page import *

% reload_ext autoreload
% autoreload 2

In [2]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Ie()

In [6]:
CPF = ["21979841870", "20048329835", "04396876670", "32252462809", "32149146851", "28482712802", 
       "02900107563", "02900107563", "17349224869", "21688805818", "13305102837", "8205179840",
       "95243178187", "10209990805", "41473019834", "03615339851", "11439066833"]

In [3]:
scpx = sistemas.Scpx(browser, usr, pwd)

In [34]:
scpx.reset_driver(scpx.driver)

NoSuchWindowException: Message: Unable to get browser


In [ ]:
ids = [("03185757874", 'id_cpf'), ("80108302164", 'id_fistel'), ("px2z1000", 'id_indicativo')]

In [ ]:
for i in ids:
    
    print("Testando Função por {}".format(i[1]))
    
    scpx.imprime_consulta(i[0], i[1])
    
    sleep(2)

Testando Função por id_cpf


In [5]:
scpx.incluir_serviço(CPF)

In [6]:
scpx.aprovar_movimento(CPF, "B")

In [8]:
scpx.licenciar_estacao(CPF)

In [9]:
scpx.consulta(CPF)

In [17]:
for cpf in CPF[8:]:
    try:
        
        scpx.imprimir_licenca(cpf)
    except UnexpectedAlertPresentException as e:
        pass
    
    input()

In [17]:
datas = ["23052018", "24052018"]
horas = ["1300", "1400"]

horarios = [(dia,hora) for dia in datas for hora in horas][2:]

print(horarios)

sistemas.abrir_agenda_prova(browser, horarios)

[('24052018', '1300'), ('24052018', '1400')]
